## Create GeoJSON files from US Census shapefiles
#### Census source: https://www2.census.gov/geo/tiger/TIGER2021/TRACT/

In [1]:
# dependencies
import geopandas as gpd
import pandas as pd
import zipfile
import os
from sqlalchemy import create_engine

# path for config file (project root folder)
import sys
sys.path.append('../../../../food-insecurity-machine-learning/')

from config import password, protocol, username, host, port, database_name

In [2]:
# create geojson files from zipped shp file
for i in range(1,79):
    try:
        # format fips codes
        if i < 10:
            n = f'0{i}'
        else:
            n = i
            
        # extract file from zip folder
        path = f'../geojson/tl_2021_{n}_tract.zip'
        file_shp = f'tl_2021_{n}_tract.shp'
        file_shx = f'tl_2021_{n}_tract.shx'
        file_dbf = f'tl_2021_{n}_tract.dbf'
        file_prj = f'tl_2021_{n}_tract.prj'
            
        with zipfile.ZipFile(path, 'r') as zf:
            zf.extract(file_shp)
            zf.extract(file_shx)
            zf.extract(file_dbf)
            zf.extract(file_prj)
            
        # convert shp file to geojson
        gdf = gpd.read_file(f'../geojson/tl_2021_{n}_tract.shp')
        geojson_str = gdf.to_json()
            
        # save geojson to file
        gdf.to_file(f'tl_2021_{n}_tract.geojson', driver='GeoJSON')            
                
        # remove shp and shx files
        os.remove(file_shp)
        os.remove(file_shx)
        os.remove(file_dbf)
        os.remove(file_prj)
            
    except Exception as e:
        print(f'Error: {str(e)}, {i}')

Error: [Errno 2] No such file or directory: '../geojson/tl_2021_03_tract.zip', 3
Error: [Errno 2] No such file or directory: '../geojson/tl_2021_07_tract.zip', 7
Error: [Errno 2] No such file or directory: '../geojson/tl_2021_14_tract.zip', 14
Error: [Errno 2] No such file or directory: '../geojson/tl_2021_43_tract.zip', 43
Error: [Errno 2] No such file or directory: '../geojson/tl_2021_52_tract.zip', 52
Error: [Errno 2] No such file or directory: '../geojson/tl_2021_57_tract.zip', 57
Error: [Errno 2] No such file or directory: '../geojson/tl_2021_58_tract.zip', 58
Error: [Errno 2] No such file or directory: '../geojson/tl_2021_59_tract.zip', 59
Error: [Errno 2] No such file or directory: '../geojson/tl_2021_61_tract.zip', 61
Error: [Errno 2] No such file or directory: '../geojson/tl_2021_62_tract.zip', 62
Error: [Errno 2] No such file or directory: '../geojson/tl_2021_63_tract.zip', 63
Error: [Errno 2] No such file or directory: '../geojson/tl_2021_64_tract.zip', 64
Error: [Errno 2] N

# Merge food access data with geojson file

In [3]:
# Connection to AWS PostgresSQL Server
rds_connection_string = f'{protocol}://{username}:{password}@{host}:{port}/{database_name}'
engine = create_engine(rds_connection_string)

In [4]:
state_df = pd.read_sql_query('select * from state', engine)
fips_codes = state_df.StateFIPS
print(fips_codes.tolist())

['01', '02', '04', '05', '06', '08', '09', '10', '11', '12', '13', '15', '16', '17', '18', '19', '20', '21', '22', '23', '24', '25', '26', '27', '28', '29', '30', '31', '32', '33', '34', '35', '36', '37', '38', '39', '40', '41', '42', '44', '45', '46', '47', '48', '49', '50', '51', '53', '54', '55', '56']


# Add food access data to geojson

In [5]:
# use this to rerun this block of code only for missing geojson merges; restore with fips_code when done
# do california (06) last
fips_temp = ['08', '09', '10', '11', '12', '13', '15', '16', '17', '18', '19', '20', '21', '22', '23', '24', '25', '26', '27', '28', '29', '30', '31', '32', '33', '34', '35', '36', '37', '38', '39', '40', '41', '42', '44', '45', '46', '47', '48', '49', '50', '51', '53', '54', '55', '56', '06']
for fips in fips_temp:
# for fips in fips_codes:
    try:    
        # read in geojson file
        gdf = gpd.read_file(f'tl_2021_{fips}_tract.geojson')
                
        # read in food access data for the state
        query = f'select * from viz_data where "StateFIPS" = ' + "'" + fips + "'"
        df = pd.read_sql_query(query, engine)
        df = df.rename(columns={'CensusTract':'GEOID'})
        
        # save CensusTract FIPS code (GEOID) in series
        tract_ids = df.GEOID
        
        # loop through GEOIDs to merge food access data to geojson files based on GEOIDs
        count = 0
        for tract in tract_ids:
            count += 1
            
            # monitor progress
            print(f'statefips: {fips}, count: {count} of {len(tract_ids)}')
            
            try:
                # merge geojson and dataframe on GEOID
                merged_gdf = gdf.merge(df, on='GEOID', how='left')
                # write merged geojson to new file
                merged_gdf.to_file(f'tl_2021_{fips}_tract_food_access.geojson', driver='GeoJSON')
            
            # print error code
            except Exception as e:
                print(f'Error: {str(e)}, state: {fips}, tract:{tract}')   
    
    # print error code
    except Exception as e:
        print(f'Error: {str(e)}, state: {fips}')

statefips: 08, count: 1 of 1242
statefips: 08, count: 2 of 1242
statefips: 08, count: 3 of 1242
statefips: 08, count: 4 of 1242
statefips: 08, count: 5 of 1242
statefips: 08, count: 6 of 1242
statefips: 08, count: 7 of 1242
statefips: 08, count: 8 of 1242
statefips: 08, count: 9 of 1242
statefips: 08, count: 10 of 1242
statefips: 08, count: 11 of 1242
statefips: 08, count: 12 of 1242
statefips: 08, count: 13 of 1242
statefips: 08, count: 14 of 1242
statefips: 08, count: 15 of 1242
statefips: 08, count: 16 of 1242
statefips: 08, count: 17 of 1242
statefips: 08, count: 18 of 1242
statefips: 08, count: 19 of 1242
statefips: 08, count: 20 of 1242
statefips: 08, count: 21 of 1242
statefips: 08, count: 22 of 1242
statefips: 08, count: 23 of 1242
statefips: 08, count: 24 of 1242
statefips: 08, count: 25 of 1242
statefips: 08, count: 26 of 1242
statefips: 08, count: 27 of 1242
statefips: 08, count: 28 of 1242
statefips: 08, count: 29 of 1242
statefips: 08, count: 30 of 1242
statefips: 08, coun